# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Reading the Price data

In [2]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [4]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA Actions

### Elite

In [5]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [6]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": tune.choice([100, 500, 1000, 2000, 5000]),
        "INITIAL_MUTATION_RATE": tune.uniform(0.05, 0.5),
        "FINAL_MUTATION_RATE": tune.uniform(0.005, 0.05),
        "ELITISM": tune.choice(np.arange(0.1, 0.6, 0.1)),
    },
    total_generations=150,
    timeout_s=60*45,
)

2024-04-01 22:23:41,563	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-01 22:23:41,616	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2024-04-01 22:23:41,640	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_9b6305cd279e1fd6.zip' (0.72MiB) to Ray cluster...
2024-04-01 22:23:41,649	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_9b6305cd279e1fd6.zip'.
2024-04-01 22:23:43,487	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-01 22:23:43,494] A new study created in memory with name: optuna


Trial name,fitness
train_770192a2,-9.09058e+06
train_7c36df5c,-7.32782e+06
train_9d1d25f9,-7.34951e+06
train_9e6c4b81,-8.24754e+06
train_aee5e695,-9.97564e+06
train_af74c724,-9.77235e+06
train_c3ee592e,-7.35901e+06
train_d7f77434,-8.491e+06
train_d874aaec,-8.06991e+06
train_f6538020,-5.9507e+06


(train pid=30904) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit


: 

In [ ]:
analysis.best_config

{'MUTPB': 1,
 'POP_SIZE': 5000,
 'INITIAL_MUTATION_RATE': 0.023464069919009116,
 'FINAL_MUTATION_RATE': 0.011522607846769213,
 'ELITISM': 0.6}

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )

### Tournament

In [ ]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [29]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": tune.choice([100, 500, 2000, 5000, 10000]),
        "CXPB": tune.uniform(0.05, 0.95),
        "INITIAL_MUTATION_RATE": tune.uniform(0.02, 0.2),
        "FINAL_MUTATION_RATE": tune.uniform(0.005, 0.02),
        "TOURNAMENT_SIZE": tune.randint(1, 50),
    },
    total_generations=300,
    timeout_s=60*6,
)

2024-04-01 21:41:04,182	INFO worker.py:1585 -- Calling ray.init() again after it has already been called.
2024-04-01 21:41:04,184	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-01 21:41:04,191] A new study created in memory with name: optuna


RuntimeError: Trying to sample a configuration from OptunaSearch, but the `metric` (fitness) or `mode` (None) parameters have not been set. Either pass these arguments when instantiating the search algorithm, or pass them to `tune.TuneConfig()`.

In [ ]:
analysis.best_config

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )